In [1]:
import os
os.chdir("./..")
%pwdb

'c:\\Users\\YNARASIM\\Desktop\\End-to-End-Machine-Learning-Project-With-MLFLOW'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [4]:
# configuration manager
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories
from mlproject import logger

class ConfigurationManger:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])
        logger.log(f"Directory: {config.root_dir} created!")

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column= schema.name
        )

        return model_trainer_config

In [5]:
import pandas as pd
import os
import joblib
from sklearn.linear_model import ElasticNet
from mlproject import logger
from mlproject.utils.common import create_directories

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        x_train = train_data.drop(self.config.target_column, axis = 1)
        x_test = test_data.drop(self.config.target_column, axis = 1)
        y_train = train_data[[self.config.target_column]]
        y_test = test_data[[self.config.target_column]]

        lr = ElasticNet(alpha = self.config.alpha, l1_ratio = self.config.l1_ratio, random_state = 42)
        lr.fit(x_train, y_train)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))


In [6]:
try:
    config = ConfigurationManger()
    model_trainer_config = config.get_model_trainer_config()
    model = ModelTrainer(model_trainer_config)
    model.train()
except Exception as e:
    raise e

[2023-08-09 17:05:27,271: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-09 17:05:27,281: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-09 17:05:27,284: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-09 17:05:27,286: INFO: common: created directory at: artifacts]
[2023-08-09 17:05:27,288: INFO: common: created directory at: artifacts/model_trainer]
